# TV Script Generation

In this project, i'll generate a [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  I'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network I'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.


In [ ]:
#load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data

In [ ]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, i first need to transform the words to ids.  In this function, i create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [ ]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    #counts = Counter(text)
    #print(counts)
    #vocab = sorted(counts, key=counts.get, reverse=True)
    #vocab_to_int = {word: num for num, word in enumerate(counts,1)}
    #print(vocab_to_int)
    #int_to_vocab = {key: value for value, key in vocab_to_int.items()}
    #print(int_to_vocab)
    # return tuple
    unique_words = set(text)
    vocab_to_int = {char:i for i, char in enumerate(unique_words)}
    int_to_vocab = {i:char for i, char in enumerate(unique_words)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation

In [ ]:
from string import punctuation

def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    #punct_dict={}
    #punctuate = ['.',',','"',';','!','?','(',')','\n','-']
    
    #for c in punctuate:
        #if c == '.':
            #punt_dict[c]='||Period||'
        #elif c == ',':
            #punt_dict[c]='Comma'
        #elif c == '\"':
            #punt_dict[c] = '||Quotation_Mark||'
        #elif c == ';':
            #punt_dict[c]= 'semicolon'
        #elif c == '!':
            #punt_dict[c]= '||Exclamation_mark||'
        #elif c == '?':
            #punt_dict[c] = '||Question_mark||'
        #elif c =='(':
            #punt_dict[c] = '||Left_Parentheses||'
        #elif c == ')':
            #punt_dict[c] = '||Right_Parentheses||'
        #elif c == '\n':
            #punt_dict[c] = '||Return||'
        #elif c == '-':
            #punt_dict[c] = '||dash||'
    #or 
    
    punct_dict = {".":"||Period||",",":"||Comma||","\"":"||Quotation_Mark||",
                      ";":"Semicolon","!":"||Exclamation_Mark||","?":"||Question_Mark||",
                       "(":"||Left_Parentheses||",")":"||Right_Paremtheses||",
                       "-":"||Dash||","\n":"||Return||"}
    return punct_dict
    

tests.test_tokenize(token_lookup)


Tests Passed


# Pre-process all the data and save it

The code cell below will pre-process all the data and save it to file, using the code in the `preprocess_and_save_data` in the `helpers.py`.

In [ ]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
The preprocessed data has been saved to disk.

In [ ]:
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network

In [ ]:
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch

def batch_data(words, sequence_length, batch_size):
    """
     Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    
    feature_tensor=[]
    target_tensor=[]
    idx = 0
    while idx<(len(words)-sequence_length):
            feature_tensor.append(words[idx:idx+sequence_length])
            target_tensor.append(words[idx+sequence_length])
            idx+=1
        
    feature_tensor = torch.from_numpy(np.array(feature_tensor))
    target_tensor = torch.from_numpy(np.array(target_tensor))
    
    data = TensorDataset(feature_tensor, target_tensor)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    
    return data_loader




### Test your dataloader 


In [ ]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). I chose to use an LSTM.
```

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        # define model layers
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(self.embedding_dim, 
                            self.hidden_dim, 
                            self.n_layers, 
                            dropout=self.dropout, 
                            batch_first=True)
        
        #self.dropout = nn.Dropout(dropout)
        
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
        
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        
        batch_size = nn_input.size(0)
        
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        
        out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1]
        
        
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        
        
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

In [ ]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # move data to GPU, if available
    if train_on_gpu:
        rnn.cuda()
        inp, target = inp.cuda(), target.cuda()
        
    
    hidden_previous = tuple([each.data for each in hidden])
    
    rnn.zero_grad()
    
    output, hidden_current = rnn(inp, hidden_previous)
    
    loss = criterion(output.squeeze(), target)
    loss.backward()
    # perform backpropagation and optimization
    clip = 5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden_current


tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training


In [ ]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

In [ ]:
# Data params
# Sequence Length
sequence_length = 9  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [ ]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
Training the neural network on the pre-processed data.  

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.466002619743347

Epoch:    1/10    Loss: 4.88364857006073

Epoch:    1/10    Loss: 4.639263223648071

Epoch:    1/10    Loss: 4.515709523677826

Epoch:    1/10    Loss: 4.515736549377442

Epoch:    1/10    Loss: 4.55597962141037

Epoch:    1/10    Loss: 4.4608212099075315

Epoch:    1/10    Loss: 4.339090788364411

Epoch:    1/10    Loss: 4.320244766235351

Epoch:    1/10    Loss: 4.257660696029663

Epoch:    1/10    Loss: 4.372026089668274

Epoch:    1/10    Loss: 4.395069370746612

Epoch:    1/10    Loss: 4.402885219097137

Epoch:    2/10    Loss: 4.195023213623724

Epoch:    2/10    Loss: 4.026999027729034

Epoch:    2/10    Loss: 3.9317459640502928

Epoch:    2/10    Loss: 3.8837180008888246

Epoch:    2/10    Loss: 3.9347124376297

Epoch:    2/10    Loss: 4.006439309597015

Epoch:    2/10    Loss: 3.9525188846588133

Epoch:    2/10    Loss: 3.8451841645240785

Epoch:    2/10    Loss: 3.8373933095932005

Epoch:    2/10    Loss: 

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


---
# Checkpoint

After running the above training cell, the model will be saved by name, `trained_rnn`.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, i'll use it to generate a new, "fake" Seinfeld TV script in this section.


In [ ]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script


In [ ]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script


pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


elaine: the mets score.

jerry: oh, no.

elaine: what do you mean?

george: well, i'm not a little bit of the same time, i think it was the last year of the incident.

jerry: what?

jerry: oh, i think it's good, so you don't want to be able to be in the bottom.

elaine: oh, that's not the same.

elaine: i think it's a lovely.

elaine: i don't know what it was. i don't know if we were going to hell this whole thing is in the city.

elaine: so?

elaine: oh.

kramer: yeah, well, you know, the guy who needs the trial.

kramer: oh.

elaine: oh, i think it's fantastic for you.

jerry: oh, i think it's the good score.

hoyt: and you know who?

jerry: yeah.

jerry: i can't get a good one.

jerry: i thought he was so uptight.

hoyt: and, i don't want to tell you, i got a lot of trouble with the sock to the end of your cell, 000.

george: i know, it's a little bit. it's not a good idea.

hoyt: and then you want to get a new saab?

jerry: i think it's a little good idea. you know, you were in the

#### Save the favorite scripts



In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()